In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Predicción de Precios de Casas en Ames, Iowa

Este notebook carga los datos, realiza preprocesamiento, entrena un modelo de XGBoost y genera
un archivo submission.csv con las predicciones para el set de test.
La métrica utilizada es el RMSE calculado sobre el logaritmo del precio.
"""

# Importamos las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print('Librerías cargadas correctamente.')

Librerías cargadas correctamente.


In [2]:
# Cargamos los datos
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

print("Dimensiones de train:", train.shape)
print("Dimensiones de test:", test.shape)

Dimensiones de train: (1460, 81)
Dimensiones de test: (1459, 80)


In [3]:
# Exploración inicial de los datos
print(train.head())
print(train.info())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [4]:
# Preprocesamiento de datos
# Concatenamos train y test para aplicar el preprocesamiento de forma uniforme
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train, test], ignore_index=True)

In [5]:
# Guardamos el Id por si se requiere luego y eliminamos la columna Id
data_ID = data['Id']
data.drop('Id', axis=1, inplace=True)

In [6]:
# Tratamos los valores nulos:
# - Para variables numéricas usamos la mediana
# - Para variables categóricas usamos el string "None"
for col in data.columns:
    if data[col].dtype == "object":
        data[col] = data[col].fillna("None")
    else:
        data[col] = data[col].fillna(data[col].median())

In [7]:
# Convertimos las variables categóricas a variables dummy
data = pd.get_dummies(data)
print("Dimensiones después del preprocesamiento:", data.shape)

Dimensiones después del preprocesamiento: (2919, 313)


In [8]:
# Separamos nuevamente los conjuntos de train y test
train_data = data[data['source_train'] == 1].copy()
test_data = data[data['source_test'] == 1].copy()

In [9]:
# Eliminamos las columnas de fuente
train_data.drop(['source_train', 'source_test'], axis=1, inplace=True)
test_data.drop(['source_train', 'source_test'], axis=1, inplace=True)

In [10]:
# Separamos la variable objetivo en train
y = train_data['SalePrice']
X = train_data.drop('SalePrice', axis=1)

In [11]:
# En test eliminamos SalePrice si existiera
if 'SalePrice' in test_data.columns:
    test_data = test_data.drop('SalePrice', axis=1)

print("Dimensiones de X:", X.shape)
print("Dimensiones de test_data:", test_data.shape)

Dimensiones de X: (1460, 310)
Dimensiones de test_data: (1459, 310)


In [12]:
# Transformamos la variable objetivo aplicando logaritmo (log1p) para suavizar su distribución
y_log = np.log1p(y)
print("Transformación del target completada.")

Transformación del target completada.


In [13]:
# Entrenamiento y evaluación del modelo
from sklearn.model_selection import cross_val_score, KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [14]:
# Definimos el modelo con algunos hiperparámetros (estos se pueden ajustar para mejorar el desempeño)
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=3, subsample=0.7, random_state=42)

In [15]:
# Evaluación mediante validación cruzada (5 folds)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = -cross_val_score(model, X, y_log, scoring="neg_mean_squared_error", cv=kf)
cv_rmse = np.sqrt(cv_scores)

print("CV RMSE scores:", cv_rmse)
print("CV RMSE promedio:", cv_rmse.mean())

CV RMSE scores: [0.12466713 0.1118332  0.15833918 0.12398089 0.1054446 ]
CV RMSE promedio: 0.12485299968840016


In [16]:
# Entrenamos el modelo final usando todo el set de entrenamiento
model.fit(X, y_log)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [17]:
# Realizamos la predicción sobre el set de test
preds_log = model.predict(test_data)

In [18]:
# Convertimos las predicciones del logaritmo al valor original
preds = np.expm1(preds_log)
print("Predicciones realizadas correctamente.")

Predicciones realizadas correctamente.


In [19]:
# Creación del archivo de submission y exportación a CSV en el directorio raíz
# Recuperamos el Id original del set de test
test_ids = test['Id']

submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": preds
})

submission.to_csv("submission.csv", index=False)
print("Archivo submission.csv creado exitosamente en el directorio raíz.")

Archivo submission.csv creado exitosamente en el directorio raíz.
